# Bottom property bias - WOA18

This notebook plots the bottom property bias of a model simulation vs. the World Ocean Atlas (2018) gridded hydrography product.

For the WOA product, we find the bottom-most data value in each grid column.

In [ ]:
import os
import datetime

In [ ]:
config = {
    "startyr": "1975",
    "endyr": "2015",
    "dora_id": "odiv-368",
    "pathPP": None,
}

MAR will pass for environment variables to the script when running via the web engine:

* `MAR_STARTYR`: Beginning year of analysis from model
* `MAR_ENDYR`: Ending year of analysis from model
* `MAR_DORA_ID`: The experiment ID in the database
* `MAR_PATHPP`: The top-level path to the post-processing experiment directory of the experiment

The block below will use values passed in by Dora but default to the values defined above in `config`. This is useful for interactive usage and debugging.

If executed from Dora, there will also be a `DORA_EXECUTE` variable that is set.

In [ ]:
for k, v in config.items():
    config[k] = (
        os.environ[f"MAR_{k.upper()}"]
        if f"MAR_{k.upper()}" in os.environ.keys()
        else v
    )

### Import Python Modules

In [ ]:
import doralite
import glob
import momlevel
import subprocess

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

from matplotlib.colors import ListedColormap, BoundaryNorm

In [ ]:
# momgrid will use a directory of pre-computed weights that is used for calculating basic area-weighted statistics later
import momgrid
os.environ["MOMGRID_WEIGHTS_DIR"] = "/nbhome/John.Krasting/grid_weights"

### Definie Local Parameters

In [ ]:
# Define some local variables. These are taken from the doralite object
# or they can be defined locally

experiment =  doralite.dora_metadata(config["dora_id"])
pathPP = experiment["pathPP"]
expName = experiment["expName"]

# Define start and end years
start = int(config["startyr"])
end = int(config["endyr"])

### Determine What Files to Load

In [ ]:
# Determine what files are needed (leave this up to the developer for flexibility)
# This is an example of what someone might do:

component = "ocean_monthly"
static = f"{component}/{component}.static.nc"
varnames = ["tob", "sob"]

chunk = "5yr"
filelists = [
    sorted(glob.glob(f"{pathPP}{component}/ts/**/{chunk}/{component}.*.{varname}.nc", recursive=True))
    for varname in varnames
]
filelist = [e for fl in filelists for e in fl]

def is_in_range(file,start,end):
    start = int(start)
    end = int(end)
    target = set(list(range(start,end+1)))
    fname = os.path.basename(file)
    times = fname.split(".")[1]
    times = times.split("-")
    times = [int(x[0:4]) for x in times]
    candidate = set(list(range(times[0],times[1]+1)))
    return len(candidate.intersection(target)) > 0

filelist = [x for x in filelist if is_in_range(x,start,end)]
staticfile = f"{pathPP}/{static}"

_ = [print(x) for x in filelist]

### DMgetting Files

Dora cannot issue calls to dmget

In [ ]:
if not "DORA_EXECUTE" in os.environ.keys():
    print("Calling dmget on files ...")
    cmd = ["dmget"]+filelist+[staticfile]
    _ = subprocess.check_output(cmd)

### Load model data and grid

In [ ]:
ds = momgrid.Gridset(filelist, force_symmetric=True, return_corners=True)
model_type = ds.model
ds = ds.data.sel(time=slice(f"{str(start).zfill(4)}-01-01",f"{str(end).zfill(4)}-12-31"))

### Load observational data (if needed)

Note that Dora only mounts `/archive`, `/nbhome`, and `/home`

In [ ]:
if "om4" in model_type:
    dsobs = "/archive/jpk/datasets/OM5/obs/WOA18/WOA18_z_35level_OM4_1080x1440_annual_v20240602.nc"
elif "om5" in model_type:
    dsobs = "/archive/jpk/datasets/OM5/obs/WOA18/WOA18_z_35level_OM5_1161x1440_annual_v20240602.nc"
else:
    raise ValueError(f"Model type {model_type} not supported")

dsobs = xr.open_dataset(dsobs).isel(time=0)
dsobs = dsobs.assign_coords({"xh": ds.xh, "yh": ds.yh})

# Get select bottom most numerical value
dsobs['tob'] = dsobs['thetao'].isel(z_l=np.isnan(dsobs['thetao']).argmax("z_l")-1)
dsobs['sob'] = dsobs['so'].isel(z_l=np.isnan(dsobs['so']).argmax("z_l")-1)

### Define some helper function for the plots

In [ ]:
def gen_levs_and_cmap(start,end,delta,cmap="RdBu_r"):
    """Generates a difference colormap centered on white"""
    boundaries = np.arange(start,end,delta)
    levels = (boundaries[0:-1] + boundaries[1:]) / 2.
    base_cmap = plt.get_cmap(cmap)
    colors = base_cmap(np.linspace(0, 1, len(levels)))
    colors[[int(len(colors) / 2) - 1]] = [1, 1, 1, 1]
    colors[[int(len(colors) / 2)]] = [1, 1, 1, 1]
    cmap = ListedColormap(colors)
    norm = BoundaryNorm(boundaries, cmap.N, clip=True)
    return (cmap, norm, boundaries)

In [ ]:
def set_annotaions(ax, title):
    _ = ax.set_xticks([])
    _ = ax.set_yticks([])
    _ = ax.text(0.0,1.06, title, weight="bold", fontsize=12, transform=ax.transAxes)
    _ = ax.text(0.0,1.02, expName, style="italic", fontsize=10, transform=ax.transAxes)
    _ = ax.text(1.0,1.05, str(starttime.values), ha="right", fontsize=8, transform=ax.transAxes)
    _ = ax.text(1.0,1.02, str(endtime.values), ha="right", fontsize=8, transform=ax.transAxes)

In [ ]:
def add_stats_box(ax, stats_str, x=0.015, y=0.8):
        # Adding the text box annotation
    props = dict(
        boxstyle="round,pad=0.3", edgecolor="black", linewidth=1.5, facecolor="white"
    )
    ax.text(
        x,
        y,
        stats_str,
        transform=ax.transAxes,
        fontsize=8,
        verticalalignment="top",
        bbox=props,
    )

In [ ]:
def calculate_stats(model,obs,areacello):
    diff = model - obs
    stats = {}
    stats["min"] = float(diff.min())
    stats["max"] = float(diff.max())
    stats = {**stats, **momgrid.xr_stats.xr_stats_2d(model,obs,ds.areacello,fmt="dict")}
    # Limit to 4 significant digits
    stats = {k:f"{v:.4g}" for k,v in stats.items()}
    # Stats string
    stats_str = str("\n").join([f"{k} = {v}" for k,v in stats.items()])
    return (stats, stats_str)

In [ ]:
def add_colorbar(fig, cb, boundaries, label):
    cbar_ax = fig.add_axes([0.16, 0.06, 0.7, 0.03])
    fig.colorbar(
        cb, cax=cbar_ax, orientation="horizontal", extend="both", ticks=boundaries[::4], label=label
    )

### Perform some calculations

In [ ]:
# Time-average the model data
model = ds
starttime = model.time[0]
endtime = model.time[-1]
model = momlevel.util.annual_average(model[["tob", "sob"]]).mean("time", keep_attrs=True).load()

# Obs data is already a climatology
obs = dsobs[["tob", "sob"]]

### Establish a Dictionary to Store Scalar Metrics

In [ ]:
stats_dict = {}
stats_dict["metadata"] = {
    "expName": str(expName),
    "created": datetime.datetime.now().isoformat(),
    "starttime": str(starttime.values),
    "endtime": str(endtime.values),
}
stats_dict["results"] = {}
stats_dict

In [ ]:
# Setup plot
for v, arange in zip(["tob", "sob"], [(-2, 2.1, 0.1), (-1, 1, 0.05)]):
    
    # Definie geolon and geolat for plotting (use corners!)
    fig = plt.figure(figsize=(10,6))
    ax = plt.subplot(1,1,1, facecolor="lightgray")

    x = ds.geolon_c
    y = ds.geolat_c
    
    # Get colormap
    cmap, norm, boundaries = gen_levs_and_cmap(*arange)
    
    # Run pcolormesh
    cb = plt.pcolormesh(x,y, model[v]-obs[v], cmap=cmap, norm=norm)
    
    # Clean up figure and add labels
    set_annotaions(ax, f"{ds[v].attrs['long_name']} relative to WOA18 (1975-2015)")
    
    # Add statistics
    stats, stats_str = calculate_stats(model[v], obs[v], ds.areacello)
    add_stats_box(ax, stats_str)
    
    # Add colorbar
    add_colorbar(fig, cb, boundaries, ds[v].attrs["units"])
    
    # Save stats
    stats_dict["results"][f"global_{v}"] = stats

### Write Stats to a File

In [ ]:
if not "DORA_EXECUTE" in os.environ.keys():
    import yaml
    yml = yaml.dump(stats_dict)
    with open('bottom_property_stats.yaml', 'w') as file:
        file.write(yml)
    file.close()
    print(yml)